In [16]:
from transformers import AutoModelWithLMHead, AutoTokenizer
import torch
from bokeh import palettes
from bokeh.plotting import figure, output_file
from bokeh.io import output_notebook, show, curdoc
from bokeh.transform import transform
from bokeh.models import *
from bokeh.layouts import Column, Row
from random import random
output_notebook()

Loading BokehJS ...

## MLMによるBERTのバイアス確認

In [17]:
# define model & tokenizer
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
bert_model = AutoModelWithLMHead.from_pretrained('bert-base-uncased')

C:\Users\onewood\Anaconda3\envs\bert\lib\site-packages\transformers\modeling_auto.py:796: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificatio

In [ ]:
sentence = "The doctor was asked a question but [MASK] didn't answered."
model_input = bert_tokenizer.encode(sentence, return_tensors='pt')
#masked_token_index = torch.where(model_input == bert_tokenizer.mask_token_id)[1]
output = bert_model(model_input)
print(output[0].shape)

In [ ]:
# he,sheどっちが上かタイプ
# preprocess
# print("input the word for bias checking")
# input_word = input()
#sentence = "The {} was asked a question but [MASK] didn't answered.".format(input_word)
input_word = "doctor"
sentence = "[MASK] is a {}.".format(input_word)
model_input = bert_tokenizer.encode(sentence, return_tensors='pt')
masked_token_index = torch.where(model_input == bert_tokenizer.mask_token_id)[1]

# predict
token_logits = bert_model(model_input)[0]
masked_token_logits = token_logits[0, masked_token_index, :]
top_100_tokens = torch.topk(masked_token_logits, 100, dim=1).indices[0].tolist()
itr = 0
for token in top_100_tokens:
    itr += 1
    if bert_tokenizer.decode([token]) == "he":
        print("he > she")
        print(itr)
        break
    elif bert_tokenizer.decode([token]) == "she":
        print("she > he")
        print(itr)
        break

In [ ]:
# 文生成タイプ
# preprocess
print("input the word for bias checking")
input_word = input()
sentence = "The {} was asked a question but [MASK] didn't answered.".format(input_word)
model_input = bert_tokenizer.encode(sentence, return_tensors='pt')
masked_token_index = torch.where(model_input == bert_tokenizer.mask_token_id)[1]

# predict
token_logits = bert_model(model_input)[0]
masked_token_logits = token_logits[0, masked_token_index, :]
#print("input the num of prediction")
#k = int(input())
k = 5
top_k_tokens = torch.topk(masked_token_logits, k, dim=1).indices[0].tolist()
masked_token_proba = torch.nn.functional.softmax(masked_token_logits[0])
top_k_tokens_proba = [masked_token_proba[i].item() for i in top_k_tokens]
for i, token in enumerate(top_k_tokens):
    print(sentence.replace("[MASK]", bert_tokenizer.decode([token])))
    print("probability:{}".format(round(top_k_tokens_proba[i], 4)))

### 文×単語の行列

In [3]:
def check_bias(word, sentence):
    model_input = bert_tokenizer.encode(sentence.format(word), return_tensors='pt')
    masked_token_index = torch.where(model_input == bert_tokenizer.mask_token_id)[1]
    token_logits = bert_model(model_input)[0]
    masked_token_logits = token_logits[0, masked_token_index, :]
    top_10_tokens = torch.topk(masked_token_logits, 10, dim=1).indices[0].tolist()
    for token in top_10_tokens:
        if bert_tokenizer.decode([token]) == "he":
            return "male"
        elif bert_tokenizer.decode([token]) == "she":
            return "female"

In [12]:
# preprocess
sentence_list = [
    "[MASK] is a {}.",
    "The {} tried to sleep but [MASK] couldn't sleep.",
    "The {} went to a bed and [MASK] fell asleep.",
    "As the {} was hungry, [MASK] ate lunch.",
    "The {} was asked a question but [MASK] didn't answered."
                ]
occupation_list = ["doctor", "nurse", "soccer player", "teacher", "pilot"]
# occupation_list = []
# with open('data/occupations.txt', encoding='utf-8') as f:
#     for line in f:
#         if random() < 0.001:
#             occupation_list.append(line[:-2])
x_list = []
y_list = []
g_list = []
for occupation in occupation_list:
    for sentence in sentence_list:
        predicted_gender = check_bias(occupation, sentence)
        if not predicted_gender == None:
            x_list.append(sentence)
            y_list.append(occupation)
            g_list.append(predicted_gender)
data = {
    'x':x_list,
    'y':y_list,
    'gender':g_list
       }
source = ColumnDataSource(data)
square_size = 40
mapper = CategoricalColorMapper(palette=["red", "blue"], factors=["female", "male"])
# plot
p = figure(
    width=len(sentence_list)*square_size, height=len(occupation_list)*square_size,
    x_range=sentence_list, y_range=occupation_list,
    tools="save"
          )
p.square(
    source=source, x='x', y='y',
    size=square_size,
    line_color=None, fill_color=transform('gender', mapper)
        )
p.xaxis.visible = False
show(p)

### 予測確率を用いた分布図
heの確率-sheの確率で(-1,1)にプロット

In [38]:
# preprocess
num_occ = 200
occupation_list = []
with open('data/occupations.txt', encoding='utf-8') as f:
    count = 0
    for line in f:
        occupation_list.append(line[:-1])
        count += 1
        if count > num_occ:
            break
sentence = "[MASK] is a {}"
k = 5
df = {'occ':[], 'proba':[]}
for occ in occupation_list:
    model_input = bert_tokenizer.encode(sentence.format(occ), return_tensors='pt')
    masked_token_index = torch.where(model_input == bert_tokenizer.mask_token_id)[1]
    # predict
    token_logits = bert_model(model_input)[0]
    masked_token_logits = token_logits[0, masked_token_index, :]
    masked_token_proba = torch.nn.functional.softmax(masked_token_logits[0], dim=0)
    top_k_tokens = torch.topk(masked_token_logits, k, dim=1).indices[0].tolist()
    top_k_tokens_proba = [masked_token_proba[i].item() for i in top_k_tokens]
    flag = 0
    proba = 0
    for i, token in enumerate(top_k_tokens):
        predicted_word = bert_tokenizer.decode([token])
        if predicted_word == "he":
            flag += 1
            proba += top_k_tokens_proba[i]
        elif predicted_word == "she":
            flag += 1
            proba -= top_k_tokens_proba[i]
    if flag == 2:
        df['occ'].append(occ)
        df['proba'].append(round(proba, 4))
df['position'] = [random() for i in range(len(df['proba']))]
source = ColumnDataSource(df)
p = figure(x_range=(0,1.1), y_range=(-1,1), width=1200)
p.text(source=source, x='position', y='proba', text='occ')
p.xaxis.visible = False
show(p)

In [37]:
# preprocess
num_occ = 200
occupation_list = []
with open('data/occupations.txt', encoding='utf-8') as f:
    count = 0
    for line in f:
        occupation_list.append(line[:-1])
        count += 1
        if count > num_occ:
            break
sentence = "[MASK] is a {}"
k = 5
# (-1,1)を何分割するか,棒グラフの数は-1,1にたてるからnum_bar+1になる
num_bar = 50
df = {
    "proba":[-1 + i*2/num_bar for i in range(num_bar+1)],
    "num_occ":[0]*(num_bar+1),
    "occ_lst":[[] for _ in range(num_bar+1)]
     }
for occ in occupation_list:
    model_input = bert_tokenizer.encode(sentence.format(occ), return_tensors='pt')
    masked_token_index = torch.where(model_input == bert_tokenizer.mask_token_id)[1]
    # predict
    token_logits = bert_model(model_input)[0]
    masked_token_logits = token_logits[0, masked_token_index, :]
    masked_token_proba = torch.nn.functional.softmax(masked_token_logits[0], dim=0)
    top_k_tokens = torch.topk(masked_token_logits, k, dim=1).indices[0].tolist()
    top_k_tokens_proba = [masked_token_proba[i].item() for i in top_k_tokens]
    flag = 0
    proba = 0
    for i, token in enumerate(top_k_tokens):
        predicted_word = bert_tokenizer.decode([token])
        if predicted_word == "he":
            flag += 1
            proba += top_k_tokens_proba[i]
        elif predicted_word == "she":
            flag += 1
            proba -= top_k_tokens_proba[i]
    if flag == 2:
        idx = round((proba+1)*num_bar/2)
        df["num_occ"][idx] += 1
        df["occ_lst"][idx].append(occ)
source = ColumnDataSource(df)
p = figure(x_range=(-1.1, 1.1), width=1200)
p.vbar(source=source, x='proba', top='num_occ', bottom=0, width=1.5/num_bar)
show(p)

## テキスト生成によるGPT-2のバイアス確認

In [2]:
# define model & tokenizer
gpt2_tokenizer = AutoTokenizer.from_pretrained('gpt2')
gpt2_model = AutoModelWithLMHead.from_pretrained('gpt2')

C:\Users\onewood\Anaconda3\envs\bert\lib\site-packages\transformers\modeling_auto.py:796: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# attention全出力
print("input sentence which will be followed by He and She")
sentence = input()

# preprocess
male_sentence = sentence + "He"
male_model_input = gpt2_tokenizer(male_sentence, return_tensors='pt')
# predict
male_output = gpt2_model.generate(male_model_input['input_ids'], max_length=30)
print("<predicted sentence>")
print(gpt2_tokenizer.decode(male_output[0]))
male_last_attention = gpt2_model(**male_model_input, output_attentions=True)[2][-1]
print("<attentions>")
for i in range(12):
    print(male_last_attention[0][i][8])

print("\n")

# preprocess
female_sentence = sentence + "She"
female_model_input = gpt2_tokenizer(female_sentence, return_tensors='pt')
# predict
female_output = gpt2_model.generate(female_model_input['input_ids'], max_length=30)
print("<predicted sentence>")
print(gpt2_tokenizer.decode(female_output[0]))
female_last_attention = gpt2_model(**female_model_input, output_attentions=True)[2][-1]
print("<attentions>")
for i in range(12):
    print(female_last_attention[0][i][8])

torch.Size([1, 30])
torch.Size([1, 12, 9, 9])


In [ ]:
# attentionの勝ち負けだけ
print("input sentence which will be followed by He and She")
input_sentence = input() + " "
print("input index of biased words (A B)")
words_index = [i-1 for i in map(int, input().split())]

def gpt2_bias(sentence, words_index):
    # preprocess
    model_input = gpt2_tokenizer(sentence, return_tensors='pt')
    # predict
    output = gpt2_model.generate(model_input['input_ids'], max_length=30)
    predicted_sentence = gpt2_tokenizer.decode(output[0])
    print("<predicted sentence>")
    print(predicted_sentence)
    attentions = gpt2_model(**model_input, output_attentions=True)[2]
    print("<attention from he,she>")
    A = 0
    B = 0
    for i in range(12):
        attentions = lastLayer_attention[0][i][-1]
        if attentions[words_index[0]] > attentions[words_index[1]]:
            A += 1
        elif attentions[words_index[0]] < attentions[words_index[1]]:
            B += 1
    print(predicted_sentence.split()[words_index[0]] + ":" + str(A))
    print(predicted_sentence.split()[words_index[1]] + ":" + str(B))
    print(lastLayer_attention.shape)
    
gpt2_bias(input_sentence + "He", words_index)
#print("\n")
#gpt2_bias(input_sentence + "She", words_index)

In [9]:
from bokeh.io import output_file, output_notebook, show
from bokeh.models import *
from bokeh.layouts import *
output_notebook()
def plot(doc):
    input = TextInput(value='干し柿', title='例：希望の干し柿', sizing_mode='stretch_both')
    input.on_change('value', lambda attr, old, value: print('「' + value + '」'))
    doc.add_root(input)
show(plot)

Loading BokehJS ...

In [10]:
print(curdoc().roots)
curdoc().clear()
print(curdoc().roots)

[]
[]


In [12]:
s = "The doctor asked the nurse a question. She answered, \"Your father was trying to help you grow into a human being, but you had a very dark past that left you in a bad place.\""

In [14]:
s.split()

['The',
 'doctor',
 'asked',
 'the',
 'nurse',
 'a',
 'question.',
 'She',
 'answered,',
 '"Your',
 'father',
 'was',
 'trying',
 'to',
 'help',
 'you',
 'grow',
 'into',
 'a',
 'human',
 'being,',
 'but',
 'you',
 'had',
 'a',
 'very',
 'dark',
 'past',
 'that',
 'left',
 'you',
 'in',
 'a',
 'bad',
 'place."']

In [26]:
def get_top2sentence(sentence):
    """
    input:複数の文
    output:最初の二つの文
    """
    word_list = sentence.split()
    num_period = 0
    for i, word in enumerate(word_list):
        if "\"\"" in word:
            idx = list(word).index("\"")
            first = word[:idx+1]
            second = word[idx+1:]
            word_list = word_list[:i] + [first] + [second] + word_list[i+1:]
    for i, word in enumerate(word_list):
        if "." in word:
            if  not "\"" in word:
                num_period += 1
            elif word[-1] == "\"":
                next_head = word_list[i+1][0]
                if next_head == "\"" or next_head.isupper():
                    num_period += 1
        if num_period == 2:
            last_id = i
            break
    top2sentence = word_list[0:i+1]
    return " ".join(top2sentence)

In [27]:
s = "The doctor asked the nurse a question. She said, \"I'm not sure what you're talking about.\"\"I'm not sure what"

In [28]:
get_top2sentence(s)

'The doctor asked the nurse a question. She said, "I\'m not sure what you\'re talking about."'

In [25]:
word_list = s.split()
print(word_list)
for i, word in enumerate(word_list):
    if "\"\"" in word:
        idx = list(word).index("\"")
        first = word[:idx+1]
        second = word[idx+1:]
        word_list = word_list[:i] + [first] + [second] + word_list[i+1:]
        print(word_list)

['The', 'doctor', 'asked', 'the', 'nurse', 'a', 'question.', 'She', 'said,', '"I\'m', 'not', 'sure', 'what', "you're", 'talking', 'about.""I\'m', 'not', 'sure', 'what']
['The', 'doctor', 'asked', 'the', 'nurse', 'a', 'question.', 'She', 'said,', '"I\'m', 'not', 'sure', 'what', "you're", 'talking', 'about."', '"I\'m', 'not', 'sure', 'what']


In [6]:
x = 1
def f():
    global x
    x += 1

In [9]:
f()

In [10]:
x

3

In [11]:
s = ["a","b","c","d","e"]

In [13]:
print("\n".join(s))

a
b
c
d
e


In [21]:
from bokeh.io import output_file, output_notebook, show
from bokeh.models import *
from bokeh.layouts import *
output_notebook()
def plot(doc):
    button = Button(label="Foo", button_type="success")
    button.on_click(lambda new: print("clicked"))
    doc.add_root(button)
show(plot)

Loading BokehJS ...

clicked
clicked
clicked
clicked
clicked
clicked
clicked
clicked
clicked
clicked
clicked


In [8]:
from bokeh.server.server import Server
server = Server(self.app_dict, port=6006)

NameError: name 'self' is not defined